### Libraries

In [52]:
import pandas as pd
from pandas import json_normalize

import requests 
import json
import os
from dotenv import load_dotenv 

import time

import re

from bs4 import BeautifulSoup as bs
from fuzzywuzzy import fuzz

C:\Users\danid\miniconda3\envs\ironhack\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


### Setting api key


In [7]:
load_dotenv()

True

In [8]:
key = os.getenv("api-key")

In [9]:
url = f'https://imdb-api.com/API/MostPopularMovies/{key}'

In [10]:
res = requests.get(url)

In [11]:
res

<Response [200]>

In [12]:
res.json()

{'items': [{'id': 'tt1630029',
   'rank': '1',
   'rankUpDown': '+1',
   'title': 'Avatar: The Way of Water',
   'fullTitle': 'Avatar: The Way of Water (2022)',
   'year': '2022',
   'image': 'https://m.media-amazon.com/images/M/MV5BYjhiNjBlODctY2ZiOC00YjVlLWFlNzAtNTVhNzM1YjI1NzMxXkEyXkFqcGdeQXVyMjQxNTE1MDA@._V1_UX128_CR0,1,128,176_AL_.jpg',
   'crew': 'James Cameron (dir.), Sam Worthington, Zoe Saldana',
   'imDbRating': '7.8',
   'imDbRatingCount': '279734'},
  {'id': 'tt9764362',
   'rank': '2',
   'rankUpDown': '-1',
   'title': 'The Menu',
   'fullTitle': 'The Menu (2022)',
   'year': '2022',
   'image': 'https://m.media-amazon.com/images/M/MV5BMzdjNjI5MmYtODhiNS00NTcyLWEzZmUtYzVmODM5YzExNDE3XkEyXkFqcGdeQXVyMTAyMjQ3NzQ1._V1_UX128_CR0,3,128,176_AL_.jpg',
   'crew': 'Mark Mylod (dir.), Ralph Fiennes, Anya Taylor-Joy',
   'imDbRating': '7.2',
   'imDbRatingCount': '210081'},
  {'id': 'tt10640346',
   'rank': '3',
   'rankUpDown': '+1',
   'title': 'Babylon',
   'fullTitle': 'Babylon 

### Web Scrapping - IMDB Netflix List

In [13]:
url_imdb = 'https://www.imdb.com/search/title/?companies=co0144901'
headers = {'Accept-Language': 'en-US,en;q=0.5'}
html = requests.get(url_imdb, headers=headers)

In [14]:
soup = bs(html.content, 'html.parser')

In [15]:
titles = soup.find_all("div", attrs = {"class":"psf-widget"})

In [16]:
title = soup.select("div h3 a")
title_ = [i.getText().strip() for i in title]
title_

["That '90s Show",
 'Wednesday',
 'You People',
 'Ginny & Georgia',
 'The Walking Dead',
 'The Pale Blue Eye',
 "That '70s Show",
 'Glass Onion',
 'All Quiet on the Western Front',
 'Fauda',
 'Breaking Bad',
 'Vikings: Valhalla',
 'Happy Valley',
 'Kaleidoscope',
 "Narvik: Hitler's First Defeat",
 'Women at War',
 'Jung_E',
 'Cobra Kai',
 'Better Call Saul',
 'Rick and Morty',
 'Mission Majnu',
 'Bullet Train',
 'Stranger Things',
 'Shahmaran',
 "Grey's Anatomy",
 'New Amsterdam',
 'Vikings',
 'Supernatural',
 'NCIS',
 'The Crown',
 'Devotion',
 'Peaky Blinders',
 'Bank of Dave',
 'Alice in Borderland',
 'Top Gun',
 'Friends',
 'Lockwood & Co',
 'RRR',
 'Teen Wolf',
 'Modern Family',
 'The Recruit',
 'Blonde',
 "Guillermo del Toro's Pinocchio",
 'You',
 'The Blacklist',
 'American Horror Story',
 'Attack on Titan',
 'Suits',
 'The Last Kingdom',
 'Shameless']

In [17]:
title[0].get('href')

'/title/tt15591076/'

In [18]:
genre = soup.select("span.genre")
genre_ = [i.getText().strip() for i in genre]
genre_

['Comedy, Drama, Romance',
 'Comedy, Crime, Fantasy',
 'Comedy, Romance',
 'Comedy, Drama',
 'Drama, Horror, Thriller',
 'Crime, Horror, Mystery',
 'Comedy, Drama, Romance',
 'Comedy, Crime, Drama',
 'Action, Drama, War',
 'Action, Drama, Thriller',
 'Crime, Drama, Thriller',
 'Action, Adventure, Drama',
 'Crime, Drama, Thriller',
 'Action, Crime, Drama',
 'Drama, History, War',
 'Drama, History',
 'Action, Adventure, Drama',
 'Action, Comedy, Drama',
 'Crime, Drama',
 'Animation, Adventure, Comedy',
 'Action, Drama, History',
 'Action, Comedy, Thriller',
 'Drama, Fantasy, Horror',
 'Adventure, Drama, Fantasy',
 'Drama, Romance',
 'Drama',
 'Action, Adventure, Drama',
 'Drama, Fantasy, Horror',
 'Action, Crime, Drama',
 'Biography, Drama, History',
 'Action, Drama, War',
 'Crime, Drama',
 'Biography, Comedy, Drama',
 'Action, Drama, Mystery',
 'Action, Drama',
 'Comedy, Romance',
 'Action, Adventure, Drama',
 'Action, Adventure, Drama',
 'Action, Drama, Fantasy',
 'Comedy, Drama, Roman

In [345]:
title = soup.find_all('h3', {'class':"lister-item-header"})[0]
title

<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">1.</span>
<a href="/title/tt15591076/">That '90s Show</a>
<span class="lister-item-year text-muted unbold">(2023– )</span>
</h3>

In [346]:
genres = soup.find_all('p', {'class':"genre"})
genres

[]

In [368]:
def scrape_imdb(n):
   
    title = []
    genre = []
    rating = []
    
    try:
        for i in range(1, n+1):
            url = f'https://www.imdb.com/search/title/?companies=co0144901&start={i}&ref_=adv_nxt'
            headers = {'Accept-Language': 'en-US,en;q=0.5'}

            response = requests.get(url, headers=headers)
            soup = bs(response.text, 'html.parser')

            # find all movie titles in the page
            titles = [a.text for a in soup.find_all('h3', {'class':"lister-item-header"})][0]

            # find genre 
            genres = [a.text for a in soup.find_all('span', {'class':"genre"})][0]

            # find type
            ratings = [a.text for a in soup.find_all('div', {'class':"inline-block ratings-imdb-rating"})][0]

            # add the titles to the results list
            title.append(titles)
            genre.append(genres)
            rating.append(ratings)
            
            #ensure they are all the same length
           # min_len = min(len(titles), len(genres), len(ratings))
           # titles = titles[:min_len]
          #  genres = genres[:min_len]
          #  ratings = ratings[:min_len]

        # create a DataFrame from the results
        df_imdb = pd.DataFrame({'title': title, 'genre': genre, 'rating': rating})
        
        
        return df_imdb
    
    except Exception as e:
        return print(f"Error scraping page {i}: {e}")




In [429]:
df_imdb = scrape_imdb(1000)

In [430]:
df_imdb.describe()

,title,genre,rating
count,1000,1000,1000
unique,1000,203,57
top,\n1.\nThat '90s Show\n(2023– )\n,"\nCrime, Drama, Mystery",\n\n7.5\n
freq,1,56,41


In [434]:
df_imdb["Title_imdb"] = df_imdb["title"].str.extract(r'\n\d+\..*\n([^\n\(\)]+)\n')

In [435]:
df_imdb["genre"] = df_imdb["genre"].str.strip('\n')

In [436]:
df_imdb["rating"] = df_imdb["rating"].str.strip('\n')

In [437]:
df_imdb["rating"].dtype

dtype('O')

In [438]:
df_imdb["rating"] = df_imdb["rating"].astype(float)

In [439]:
df_imdb.sample(5)

,title,genre,rating,Title_imdb
864,\n865.\nLove in the Villa\n(2022)\n,"Comedy, Romance",5.4,Love in the Villa
110,\n111.\nSherlock\n(2010–2017)\n,"Crime, Drama, Mystery",9.1,Sherlock
627,\n628.\nThe Unforgivable\n(2021)\n,"Crime, Drama",7.1,The Unforgivable
752,\n753.\nBlack Crab\n(2022)\n,"Action, Adventure, Drama",5.7,Black Crab
469,\n470.\nI Used to Be Famous\n(2022)\n,"Comedy, Drama, Music",6.6,I Used to Be Famous


In [440]:
df_imdb.drop(['title'], axis=1, inplace=True)

In [441]:
df_imdb = df_imdb.reindex(columns=['Title_imdb', 'genre', 'rating'])

In [442]:
df_imdb.sample()

,Title_imdb,genre,rating
298,Your Place or Mine,"Comedy, Romance",7.1


In [443]:
df_imdb['Title_imdb'].describe()

count                            996
unique                           986
top       Avatar: The Last Airbender
freq                               2
Name: Title_imdb, dtype: object

In [444]:
df_imdb.shape

(1000, 3)

### CSV File - My Netflix viewing list 

In [445]:
with open("data/netflix_viewing-history.csv") as f:
    print(f)

<_io.TextIOWrapper name='data/netflix_viewing-history.csv' mode='r' encoding='cp1252'>


In [446]:
df_net = pd.read_csv("data/netflix_viewing-history.csv")

In [447]:
df_net.describe()

,Title,Date
count,6177,6177
unique,5857,1177
top,: Episode 1,27/11/2022
freq,7,45


In [448]:
df_net.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6177 entries, 0 to 6176
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Title   6177 non-null   object
 1   Date    6177 non-null   object
dtypes: object(2)
memory usage: 96.6+ KB


In [449]:
df_net['Date'] = pd.to_datetime(df_net['Date'], dayfirst=True)

In [450]:
df_net["Title_net"] = df_net["Title"].str.extract(r"^([^:]+)")

In [451]:
df_net.sample(5)

,Title,Date,Title_net
1670,Mo: Season 1: Tombstone,2022-04-09,Mo
3110,Kim's Convenience: Season 2: Sneak Attack,2021-06-28,Kim's Convenience
4346,How I Met Your Mother: Season 2: Aldrin Justice,2020-09-22,How I Met Your Mother
747,I Think You Should Leave with Tim Robinson: Se...,2019-05-16,I Think You Should Leave with Tim Robinson
4164,Pan,2020-11-15,Pan


In [452]:
df_net['Date'].dtype

dtype('<M8[ns]')

In [453]:
df_net['Title_net'].describe()

count          6177
unique          832
top       Star Trek
freq            488
Name: Title_net, dtype: object

In [454]:
df_net.drop_duplicates(subset="Title_net", keep='first', inplace=True)

In [455]:
df_net['Title_net'].describe()

count               832
unique              832
top       The Godfather
freq                  1
Name: Title_net, dtype: object

In [456]:
df_net.sample(5)

,Title,Date,Title_net
5276,Dark Shadows,2019-10-23,Dark Shadows
2235,Playing with Fire,2022-09-01,Playing with Fire
161,Amy Schumer Growing,2022-03-30,Amy Schumer Growing
2607,Adventure Beast: Nature Hates Jerks,2021-10-31,Adventure Beast
5878,The Lonely Island Presents: The Unauthorized B...,2019-05-25,The Lonely Island Presents


In [457]:
df_net.drop(['Title'], axis=1, inplace=True)

### Merge IMDB and Netflix Lists 

In [458]:
def merge_dfs(df_imdb, df_net, on='Title2'):
    if on in df_imdb.columns and on in df_net.columns:
        df_net = df_net.rename(columns={ on:f'{on}_right'})
        #on = f'{on}_right'
    df_merged = pd.merge(df_imdb, df_net, on=on)
    return df_merged


In [459]:
df_net.describe()

C:\Users\danid\AppData\Local\Temp\ipykernel_14408\3941314626.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df_net.describe()


,Date,Title_net
count,832,832
unique,544,832
top,2020-11-14 00:00:00,The Godfather
freq,7,1
first,2017-07-30 00:00:00,NaN
last,2023-12-01 00:00:00,NaN


In [460]:
df_imdb['Title_imdb'].describe()

count                            996
unique                           986
top       Avatar: The Last Airbender
freq                               2
Name: Title_imdb, dtype: object

In [461]:
merge_dfs = pd.merge(df_net, df_imdb, left_on='Title_net', right_on='Title_imdb', how='left')

In [462]:
merge_dfs.sample()

,Date,Title_net,Title_imdb,genre,rating
349,2022-10-29,Happy Death Day 2U,NaN,NaN,NaN


In [463]:
df_imdb['Title_imdb'].unique

<bound method Series.unique of 0        That '90s Show
1             Wednesday
2            You People
3       Ginny & Georgia
4      The Walking Dead
             ...       
995              Mirage
996            Mudbound
997        The Takeover
998     Despicable Me 2
999                 NaN
Name: Title_imdb, Length: 1000, dtype: object>

In [464]:
df_net['Title_net'].unique

<bound method Series.unique of 0                     The Godfather
1                            Trolls
2                      VINLAND SAGA
3              Arrested Development
4                     Puss in Boots
                   ...             
6161    Tidying Up with Marie Kondo
6173                     Safe House
6174                     Win It All
6175                    The Cobbler
6176     The Fundamentals of Caring
Name: Title_net, Length: 832, dtype: object>

In [465]:
merge_dfs.sample(10)

,Date,Title_net,Title_imdb,genre,rating
590,2021-08-01,Equinox,NaN,NaN,NaN
714,2020-02-23,Game Night,NaN,NaN,NaN
215,2020-05-23,The Adjustment Bureau,NaN,NaN,NaN
382,2022-07-25,The Best of Netflix Is a Joke,NaN,NaN,NaN
289,2018-04-11,Day of the Dead,NaN,NaN,NaN
562,2021-03-13,ARQ,NaN,NaN,NaN
566,2021-06-03,Nerve,NaN,NaN,NaN
605,2020-12-20,Christmas Made to Order,NaN,NaN,NaN
411,2022-03-26,Black Crab,Black Crab,"Action, Adventure, Drama",5.7
225,2020-04-28,The House Bunny,NaN,NaN,NaN


In [466]:
merged = merge_dfs.drop_duplicates(subset = "Title_imdb")

In [467]:
merged

,Date,Title_net,Title_imdb,genre,rating
0,2023-01-29,The Godfather,NaN,NaN,NaN
3,2023-01-29,Arrested Development,Arrested Development,Comedy,8.7
7,2023-01-28,Wednesday,Wednesday,"Comedy, Crime, Fantasy",8.2
9,2023-01-14,The Pale Blue Eye,The Pale Blue Eye,"Crime, Horror, Mystery",6.6
10,2023-01-08,1899,1899,"Drama, Mystery",7.4
...,...,...,...,...,...
809,2019-03-22,The Dirt,The Dirt,"Biography, Comedy, Drama",7.0
810,2019-03-18,Triple Frontier,Triple Frontier,"Action, Thriller",6.4
816,2019-04-03,Serenity,Serenity,"Drama, Mystery, Thriller",5.4
820,2019-02-27,Velvet Buzzsaw,Velvet Buzzsaw,"Horror, Mystery, Thriller",5.7


In [469]:
merged['Title_imdb'].describe()

count                      243
unique                     243
top       Arrested Development
freq                         1
Name: Title_imdb, dtype: object

In [473]:
merged['Title_imdb'].isna().sum()

1

In [474]:
merged.to_csv('data/clean-and-merged_movies.csv')
